In [2]:
# imports for code 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [29]:
# urls and dataframes
url_test ='https://raw.githubusercontent.com/matankleiner/ProjectB/master/data/test/test.csv'
url_pred = 'https://raw.githubusercontent.com/matankleiner/Identify-Known-Sites-in-Photo-Album/master/feature_extraction/predicted_class_embedded_test.csv'
url_train = 'https://raw.githubusercontent.com/matankleiner/Identify-Known-Sites-in-Photo-Album/master/data/train/train.csv' 
test_df = pd.read_csv(url_test) 
pred_df = pd.read_csv(url_pred)
train_df = pd.read_csv(url_train)

In [19]:
test_df

,id,landmarks
0,e324e0f3e6d9e504,0
1,d9e17c5f3e0c47b3,0
2,1a748a755ed67512,0
3,537bf9bdfccdafea,0
4,13f4c974274ee08b,0
...,...,...
117222,e351c3e672c25fbd,190441
117223,5426472625271a4d,0
117224,7b6a585405978398,0
117225,d885235ba249cf5d,0


In [20]:
pred_df = pred_df.drop("Unnamed: 0", axis=1)
pred_df = pred_df.rename(columns={"0": "prediction"})
pred_df.insert(0, "id", test_df["id"], True) 
pred_df

,id,prediction
0,e324e0f3e6d9e504,42422
1,d9e17c5f3e0c47b3,14968
2,1a748a755ed67512,164193
3,537bf9bdfccdafea,136675
4,13f4c974274ee08b,25369
...,...,...
117222,e351c3e672c25fbd,23777
117223,5426472625271a4d,54785
117224,7b6a585405978398,21500
117225,d885235ba249cf5d,162403


In [23]:
s = test_df["landmarks"] == pred_df["prediction"]

In [24]:
s[s].index

Int64Index([], dtype='int64')

In [35]:
train_df.loc[378345]

id             6433c80f2d907e7a
landmark_id               47663
Name: 378345, dtype: object

In [36]:
train_df.loc[1481565]

id             d496ccee421f2048
landmark_id              190441
Name: 1481565, dtype: object

In [37]:
train_df.loc[184639]

id             91d87f15520299a2
landmark_id               23777
Name: 184639, dtype: object

In [38]:
train_df.loc[184498]

id             0f28f3fb43a14406
landmark_id               23777
Name: 184498, dtype: object

In [39]:
train_df.loc[442828]

id             81720579d3871be9
landmark_id               56062
Name: 442828, dtype: object